# NLP - Text Processing

When doing natural language machine learning, you need to be able to take text and convert it into numerical form for analysis. As a a part of my data science nanodegree I want to note down all of the ways that you can transform text into data that you can use for machine learning.

When it comes to natural language processing there are generally two steps. Text docuement `cleaning`, so that we are only left with strings that have the maximal amount of "meaning". And `feature extraction`, where we take these words and convert them to a numerical form for statistical analysis.

In [1]:
# Import packages
import requests
import re

from bs4 import BeautifulSoup
from nltk import pos_tag, ne_chunk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

# Cleaning

When using documents, and especially those that you get off of the internet. You will need to take the format of the document and do some rudimentary changes so that you can start to process it. 

This cleaning process can be tedious but it is necessary in order to do the fun parts of data analysis.

Here, I use an example of how we can clean the data that comes from a webpage

In [2]:
# Fetch a web page
r = requests.get("https://www.udacity.com/courses/all")
print(r.text)

<!DOCTYPE html><html><head>
  <meta charset="utf-8">
  <script type="text/javascript" class="ng-star-inserted">window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s.console&&console.log(t)}catch(n){}}var o,i=t("ee"),a=t(15),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&(s.nrDev=!0))}catch(c){}s.nrDev&&i.on("internal-error",function(t){r(t.stack)}),s.dev&&i.on("fn-err",function(t,n,e){r(e.stack)}),s.dev&&(r("NR AGENT IN DEVELOPMENT MODE"),r("flags: "+a(s,function(t,n){return t}).join(", ")))},{}],2:[function(t,n,e){function r(t,n,e,r,s){try{p?p-=1:o(s||new UncaughtException(t,n,e

You can see above that by getting this webpage that we get far more information than we are able to process. We need to parse all of this using `BeautifulSoup`

In [3]:
soup = BeautifulSoup(r.text, "html5lib")

print(soup.get_text())


  
  window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s.console&&console.log(t)}catch(n){}}var o,i=t("ee"),a=t(15),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&(s.nrDev=!0))}catch(c){}s.nrDev&&i.on("internal-error",function(t){r(t.stack)}),s.dev&&i.on("fn-err",function(t,n,e){r(e.stack)}),s.dev&&(r("NR AGENT IN DEVELOPMENT MODE"),r("flags: "+a(s,function(t,n){return t}).join(", ")))},{}],2:[function(t,n,e){function r(t,n,e,r,s){try{p?p-=1:o(s||new UncaughtException(t,n,e),!0)}catch(f){try{i("ierr",[f,c.now(),!0])}catch(d){}}return"function"==typeof u&&u.apply(this,a(argumen

Doing this cleans things up a little bit but we are still not really entirely interested on only the entire page.

If we are looking for a specific kind of tab like a `div` with the class `course-summary-card` then we can type the following:

In [4]:
summaries = soup.find_all("div", class_="course-summary-card")

# Take a look at one summary
summaries[0]

<div _ngcontent-sc163="" class="course-summary-card row row-gap-medium catalog-card nanodegree-card ng-star-inserted"><ir-catalog-card _ngcontent-sc163="" _nghost-sc166=""><div _ngcontent-sc166="" class="card-wrapper is-collapsed"><div _ngcontent-sc166="" class="card__inner card mb-0"><div _ngcontent-sc166="" class="card__inner--upper"><div _ngcontent-sc166="" class="image_wrapper hidden-md-down"><a _ngcontent-sc166="" href="/course/ai-product-manager-nanodegree--nd088"><!----><div _ngcontent-sc166="" class="image-container ng-star-inserted" style="background-image:url(https://d20vrrgs8k4bvw.cloudfront.net/images/degrees/nd088/nd-card.png);"><div _ngcontent-sc166="" class="image-overlay"></div></div></a><!----></div><div _ngcontent-sc166="" class="card-content"><!----><span _ngcontent-sc166="" class="tag tag--new card ng-star-inserted">New</span><!----><div _ngcontent-sc166="" class="category-wrapper"><span _ngcontent-sc166="" class="mobile-icon"></span><!----><h4 _ngcontent-sc166="" c

We can get the course title from the h3 header

In [5]:
summaries[0].select_one("h3").get_text().strip()

'AI Product Manager'

In [6]:
summaries[0].select_one("h4").get_text().strip()

'School of Artificial Intelligence'

We can see that we can easily access different parts of a webpage by using the HTML formatting of a page.

## Normalisation

It's worth noting that even if you have plain human text, words that mean the same thing can be differentiated based on the context:

* Car
* car
* CAR

All of these words mean, "car" but they are different because of the capitalization. We want to make sure that when we do machine learning that we remove things of this nature.

In [7]:
doc = "The first time you see The Second Reniassance it may look boring. Look at it at least twice \
and definitely watch part 2. It will change your view of the matrix. Are the human people the ones \
who started the war? Is AI a bad thing?"

doc_lower = doc.lower()

doc_lower

'the first time you see the second reniassance it may look boring. look at it at least twice and definitely watch part 2. it will change your view of the matrix. are the human people the ones who started the war? is ai a bad thing?'

We can see here already that we can use string functions to bein to normalize a text.

Notice a swell that there is punctuation that we aren't necessarily interested in either, we can also try to remove those.

In [8]:
doc_rm_punc = re.sub(r"[^a-zA-Z0-9]", " ", doc_lower)

doc_rm_punc

'the first time you see the second reniassance it may look boring  look at it at least twice and definitely watch part 2  it will change your view of the matrix  are the human people the ones who started the war  is ai a bad thing '

## Tokenization

Next, we need to take the text that we have here and convert it into a list of strings or `tokens`. This can be done by splitting a string by " " for instance.

In [9]:
token = doc_rm_punc.split()

token

['the',
 'first',
 'time',
 'you',
 'see',
 'the',
 'second',
 'reniassance',
 'it',
 'may',
 'look',
 'boring',
 'look',
 'at',
 'it',
 'at',
 'least',
 'twice',
 'and',
 'definitely',
 'watch',
 'part',
 '2',
 'it',
 'will',
 'change',
 'your',
 'view',
 'of',
 'the',
 'matrix',
 'are',
 'the',
 'human',
 'people',
 'the',
 'ones',
 'who',
 'started',
 'the',
 'war',
 'is',
 'ai',
 'a',
 'bad',
 'thing']

However, this is a faily naive way to perform this operation. Instead, we can use standard natural language toolkits in order to tokenize things.

In [10]:
try: 
    tokens = word_tokenize(doc)
except LookupError:
    import nltk
    nltk.download('punkt')

tokens

['The',
 'first',
 'time',
 'you',
 'see',
 'The',
 'Second',
 'Reniassance',
 'it',
 'may',
 'look',
 'boring',
 '.',
 'Look',
 'at',
 'it',
 'at',
 'least',
 'twice',
 'and',
 'definitely',
 'watch',
 'part',
 '2',
 '.',
 'It',
 'will',
 'change',
 'your',
 'view',
 'of',
 'the',
 'matrix',
 '.',
 'Are',
 'the',
 'human',
 'people',
 'the',
 'ones',
 'who',
 'started',
 'the',
 'war',
 '?',
 'Is',
 'AI',
 'a',
 'bad',
 'thing',
 '?']

Using `nltk`, we can keep more of the punctuation in the correct places. 

You can even split paragraphs into sentences using `sent_tokenize` and other tokenizers, such as ones that help with tweets. (Make sure to remember this for the project.) Make sure that you search and find ones appropriate to your use case!

## Stop Word Removal

Now that we have our tokens, we want to get rid of "stop words" which are words in a language that don't have a lot of meaning. They are typically very common like "are" or "the". If we remove these words then you should still be able to infer the meaning of a sentence without them. 

In [11]:
try:
    stopwords.words("english")
except LookupError:
    nltk.download("stopwords")
    
stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

There are quite a few stopwords which is why it'S good to have packages like this to remember these things for us. We can even start to do analysis in other languages as well. 

In [12]:
tokens_wo_sw = [w.lower() for w in tokens if w.lower() not in stopwords.words("english")]

tokens_wo_sw

['first',
 'time',
 'see',
 'second',
 'reniassance',
 'may',
 'look',
 'boring',
 '.',
 'look',
 'least',
 'twice',
 'definitely',
 'watch',
 'part',
 '2',
 '.',
 'change',
 'view',
 'matrix',
 '.',
 'human',
 'people',
 'ones',
 'started',
 'war',
 '?',
 'ai',
 'bad',
 'thing',
 '?']

We can see now how the list of tokens has been cut down to something more reasonable and manageable. 

## PoS Tagging

We might also be interested in what part of speech a sentence is. We can use parts of speech tagging for this.

In [13]:
sentence = word_tokenize("I always lie down to tell a lie.")
pos_tag(sentence)

[('I', 'PRP'),
 ('always', 'RB'),
 ('lie', 'VBP'),
 ('down', 'RP'),
 ('to', 'TO'),
 ('tell', 'VB'),
 ('a', 'DT'),
 ('lie', 'NN'),
 ('.', '.')]

We can see here how the two `lie` words were both classified as a verb and as a noun. This means now that we might not mix up the two words, where we might in a more naive setting.

## Named Entity Recognition

It's also possible that you get various nouns but you want to know what these nouns are. You can use Named Entities to recognise that.

In [14]:
try:
    ne_chunk(pos_tag(word_tokenize("Antonio join Udacity Inc. in California.")))
except LookupError:
    print("You need to install ghost script to see the output of this function")

## Stemming

Note as well that not only can you use capitalization to make the same word appear in different forms, the same can also appear differently based on things like tense as well as other things. i.e. "branching", "branched", etc.

These examples all belong to the same stem "branch". Doing this kind of thing can be a quick and easy way to make sure that words with the same meaning dont get counted multiple times. But once again, this method is a little naive and cannot fix for words like "Ran" and "Running".

Note as well that there are multiple different stemmers that do different things so make sure you pay attention to that.

In [15]:
stemmed = [PorterStemmer().stem(w) for w in tokens]

stemmed

['the',
 'first',
 'time',
 'you',
 'see',
 'the',
 'second',
 'reniass',
 'it',
 'may',
 'look',
 'bore',
 '.',
 'look',
 'at',
 'it',
 'at',
 'least',
 'twice',
 'and',
 'definit',
 'watch',
 'part',
 '2',
 '.',
 'It',
 'will',
 'chang',
 'your',
 'view',
 'of',
 'the',
 'matrix',
 '.',
 'are',
 'the',
 'human',
 'peopl',
 'the',
 'one',
 'who',
 'start',
 'the',
 'war',
 '?',
 'Is',
 'AI',
 'a',
 'bad',
 'thing',
 '?']

Some of the conversions make sense and others less so.

## Lemmatization

Instead of simply stemming words, we can use a dictionary to get words back to their necessary roots. This is a more conputationally heavy approach and requires that the words have been predefined in order for this to work

In [16]:
lemmed = [WordNetLemmatizer().lemmatize(w) for w in tokens]

lemmed

['The',
 'first',
 'time',
 'you',
 'see',
 'The',
 'Second',
 'Reniassance',
 'it',
 'may',
 'look',
 'boring',
 '.',
 'Look',
 'at',
 'it',
 'at',
 'least',
 'twice',
 'and',
 'definitely',
 'watch',
 'part',
 '2',
 '.',
 'It',
 'will',
 'change',
 'your',
 'view',
 'of',
 'the',
 'matrix',
 '.',
 'Are',
 'the',
 'human',
 'people',
 'the',
 'one',
 'who',
 'started',
 'the',
 'war',
 '?',
 'Is',
 'AI',
 'a',
 'bad',
 'thing',
 '?']

# Feature Extraction

Using a combination of methods from the previous section, we can clean and normalize words for use. However, we can't use this data in machine learning just yet because machine learning requires numerical data. So what do we do?

Well, there are various methods we can pick up depending on what we are interested in. From simple bag-of-words to one-hot encoding, where we simply count the number of words or their positions to more advanced methods that try and retain the meanings in sentences, there are a lot of methods to choose from.

## Bag of Words

This is the most basic way to do feature extraction, you basically create a document x word matrix where the rows are your documents and the columns are the words that are in these documents. A value is then assigned to each position based on whether they appear in the document or not or even the frequency at which these words appear.

Note that these words can appear in any order.

![Bag of Words](bagofwords.png)

We can start finding similarities by comparing dot products of the vectors in the row/column space of the matrix

## TF-IDF

Limitation of BoW: treats each word as if it is equally important to another word. We can fix this by summing up the number of times the word appears in the corpus and dividing the numbers by this value.

![](tfidf.png)

## Word Embeddings

If we try to think about placing words in a high-dimensional space, then we want the words to be as such that if they appear closer to each other then they are similar and that even adding or subtracting words would give us logical answers. i.e. king - man + woman = queen